In [ ]:
%%capture
%pip install -U bitsandbytes
%pip install -U transformers
%pip install -U accelerate

In [ ]:
from huggingface_hub import login

login(token = HUGGINGFACE_TOKEN) # create an access token in hugging face with token type as "write"

In [ ]:
import os
# Disable torch dynamo completely
os.environ["PYTORCH_DISABLE_PYTORCH_LIGHTNING_DYNAMO"] = "1"
os.environ["TORCH_COMPILE_DISABLE"] = "1"
os.environ["TORCHDYNAMO_DISABLE"] = "1"

In [ ]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig, AutoConfig

modelName = "google/gemma-2-2b-it"

bnbConfig = BitsAndBytesConfig(
    load_in_4bit = True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16,
)

tokenizer = AutoTokenizer.from_pretrained(modelName)

model = AutoModelForCausalLM.from_pretrained(
    modelName,
    device_map = "auto",
    quantization_config=bnbConfig
)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
from IPython.display import Markdown, display
import torch._dynamo

system =  "You are a legal assistant. You help the user with Indian legal law related queries."
user = "What is the difference between a petition and a plaint in Indian law?"

prompt = f"System: {system} \n User: {user} \n AI: "

inputs = tokenizer(prompt, return_tensors='pt', padding=True, truncation=True).to("cuda")

outputs = model.generate(**inputs, max_length=500, num_return_sequences=1)

text = tokenizer.decode(outputs[0], skip_special_tokens=True)

Markdown(text.split("AI:")[1])

 
  A petition and a plaint are both documents filed in a court of law in India, but they have distinct purposes and structures. 

**Here's a breakdown of the differences:**

**Petition:**

* **Definition:** A petition is a formal document filed by a party (plaintiff or petitioner) in a court of law to seek a specific relief or order. 
* **Purpose:** To initiate legal proceedings and seek a remedy for a legal wrong or violation.
* **Content:** Typically includes a detailed statement of facts, legal arguments, and the relief sought. 
* **Examples:** 
    * A petition for divorce.
    * A petition for maintenance.
    * A petition for custody of a child.

**Plaint:**

* **Definition:** A plaint is a document filed by a plaintiff in a civil suit. It is essentially a complaint that outlines the plaintiff's claim against the defendant.
* **Purpose:** To initiate a civil suit and seek a remedy for a civil wrong or breach of contract.
* **Content:**  Includes a detailed statement of facts, legal arguments, and the relief sought. 
* **Examples:** 
    * A plaint for breach of contract.
    * A plaint for recovery of debt.
    * A plaint for damages.

**Key Differences:**

* **Nature of proceedings:** A petition is used in a wide range of legal proceedings, while a plaint is specifically used in civil suits.
* **Parties involved:** A petition can be filed by either a plaintiff or a defendant, while a plaint is always filed by the plaintiff.
* **Focus:** A petition focuses on seeking a specific relief or order, while a plaint focuses on initiating a civil suit and seeking a remedy for a civil wrong.

**In summary:**

* **Petition:** A general term for a document filed in court seeking a specific relief.
* **Plaint:** A specific document filed in a civil suit to initiate a legal action.


**Note:** This information is for general knowledge and should not be considered legal advice. It is always best to consult with a qualified legal professional for specific legal advice. 